### Final Project Course 3

In [ ]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup
from datetime import datetime

WIKI_URL = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
EXCHANGE_CSV = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
CSV_OUTPUT = "./Largest_banks_data.csv"
DB_NAME = "Banks.db"
TABLE_NAME = "Largest_banks"
LOG_FILE = "code_log.txt"

# Task 1
def log_progress(message):
    with open(LOG_FILE, "a") as f:
        f.write(f"{datetime.now()} : {message}\n")

# Task 2
def extract():
    response = requests.get(WIKI_URL)
    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("table", {"class": "wikitable sortable mw-collapsible"})
    df = pd.read_html(str(tables))[0]
    print(df.columns)
    df = df[['Bank name', 'Market cap (US$ billion)']]
    df.columns = ['Name', 'MC_USD_Billion']
    return df

# Task 3
def transform(df):
    exchange_df = pd.read_csv(EXCHANGE_CSV)
    rates = dict(zip(exchange_df['Currency'], exchange_df['Rate']))
    df["MC_GBP_Billion"] = (df["MC_USD_Billion"] * rates["GBP"]).round(2)
    df["MC_EUR_Billion"] = (df["MC_USD_Billion"] * rates["EUR"]).round(2)
    df["MC_INR_Billion"] = (df["MC_USD_Billion"] * rates["INR"]).round(2)
    return df

# Task 4
def load_to_csv(df):
    df.to_csv(CSV_OUTPUT, index=False)

# Task 5
def load_to_db(df):
    conn = sqlite3.connect(DB_NAME)
    df.to_sql(TABLE_NAME, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

# Task 6
def run_queries():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    result = cursor.execute(f"SELECT * FROM {TABLE_NAME} LIMIT 5").fetchall()
    conn.close()
    return result

# Main ETL
log_progress("Start ETL process")
log_progress("Extracting data...")
df_extracted = extract()
log_progress("Extraction complete")

log_progress("Transforming data...")
df_transformed = transform(df_extracted)
log_progress("Transformation complete")

log_progress("Saving to CSV...")
load_to_csv(df_transformed)
log_progress("Saved to CSV")

log_progress("Loading to DB...")
load_to_db(df_transformed)
log_progress("Loaded to DB")

log_progress("Running test query on DB...")
query_results = run_queries()
log_progress("Query execution complete")

log_progress("ETL process complete")
print("✅ Pipeline executed. Sample DB rows:\n", query_results)



/tmp/ipykernel_369366/2530951343.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables))[0]


Index(['Rank', 'Bank name', 'Market cap (US$ billion)'], dtype='object')
✅ Pipeline executed. Sample DB rows:
 [('JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71), ('Bank of America', 231.52, 185.22, 215.31, 19204.58), ('Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75), ('Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41), ('HDFC Bank', 157.91, 126.33, 146.86, 13098.63)]


In [ ]:
d = extract()
d

Index(['Rank', 'Bank name', 'Market cap (US$ billion)'], dtype='object')


/tmp/ipykernel_369366/2530951343.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables))[0]


,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [ ]:
d = transform(d)
d

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39


In [ ]:
df_sorted = d.sort_values(by='MC_USD_Billion', ascending=False).reset_index(drop=True)
fifth_bank_eur = df_sorted.loc[4, 'MC_EUR_Billion']
print(fifth_bank_eur)



146.86


In [ ]:
conn = sqlite3.connect("Banks.db")
avg = conn.execute("SELECT AVG(MC_GBP_Billion) FROM Largest_banks").fetchone()
print(avg)


(151.987,)
